In [1]:
import datetime
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from threading import Thread, Event

class IBKRClient(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = pd.DataFrame(columns=["Close"])  # Store price data
        self.nextValidOrderId = None
        self.position = 0  # Number of shares held
        self.cash = 10000  # Initial cash balance for simulation
        self.stock_value = 0
        self.net_assets = self.cash  # Net assets (cash + stock value)
        self.realtime_price = None
        self.order_ready = Event()

    def nextValidId(self, orderId):
        self.nextValidOrderId = orderId
        self.order_ready.set()

    def error(self, reqId, errorCode, errorString):
        print(f"Error: {reqId}, Code: {errorCode}, Msg: {errorString}")

    def realtimeBar(self, reqId, time, open_, high, low, close, volume, wap, count):
        print(f"Real-time price: {close}")
        # Update the data with the new price
        self.data = self.data.append({"Close": close}, ignore_index=True)
        self.realtime_price = close
        self.execute_strategy()

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. Id: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, AvgFillPrice: {avgFillPrice}")
        if status == "Filled":
            if self.position > 0:
                self.cash -= filled * avgFillPrice
            else:
                self.cash += filled * avgFillPrice
            self.update_net_assets()

    def position(self, account, contract, position, avgCost):
        self.position = position
        print(f"Position: {position} shares at an average cost of {avgCost}")

    def positionEnd(self):
        print("End of Position Data")

    def execute_strategy(self):
        sma50 = self.data['Close'].rolling(window=50).mean().iloc[-1] if len(self.data) >= 50 else None
        sma200 = self.data['Close'].rolling(window=200).mean().iloc[-1] if len(self.data) >= 200 else None

        if sma50 and sma200:
            print(f"SMA50: {sma50}, SMA200: {sma200}")
            if sma50 > sma200 and self.position == 0:
                print("Buy signal detected")
                self.place_order("BUY", 10, "MKT")
            elif sma50 < sma200 and self.position > 0:
                print("Sell signal detected")
                self.place_order("SELL", self.position, "MKT")

    def place_order(self, action, quantity, order_type):
        self.order_ready.wait()
        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = order_type
        self.placeOrder(self.nextValidOrderId, self.contract, order)
        self.nextValidOrderId += 1
        self.order_ready.clear()

    def update_net_assets(self):
        self.stock_value = self.position * self.realtime_price if self.realtime_price else 0
        self.net_assets = self.cash + self.stock_value
        print(f"Net Assets: {self.net_assets}, Cash: {self.cash}, Stock Value: {self.stock_value}")

def create_contract(symbol, sec_type, exchange, currency):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exchange
    contract.currency = currency
    return contract

def main():
    # Initialize IBKR client
    client = IBKRClient()
    client.connect("127.0.0.1", 4002, clientId=1)  # Port 7497 for paper trading

    # Run the client in a separate thread
    client_thread = Thread(target=client.run)
    client_thread.start()

    # Wait until next valid order ID is received
    client.order_ready.wait()

    # Create contract
    client.contract = create_contract("AAPL", "STK", "SMART", "USD")

    # Request real-time market data
    client.reqRealTimeBars(1, client.contract, 5, "TRADES", True, [])

    try:
        while True:
            # Continuously update net assets
            client.update_net_assets()
            Event().wait(5)  # Wait 5 seconds before the next update
    except KeyboardInterrupt:
        print("Stopping the trading client...")
    finally:
        client.disconnect()
        client_thread.join()

if __name__ == "__main__":
    main()


Error: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm
Error: -1, Code: 2107, Msg: HMDS data farm connection is inactive but should be available upon demand.ushmds
Error: -1, Code: 2158, Msg: Sec-def data farm connection is OK:secdefil
Net Assets: 10000, Cash: 10000, Stock Value: 0
Error: -1, Code: 2106, Msg: HMDS data farm connection is OK:ushmds
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Net Assets: 10000, Cash: 10000, Stock Value: 0
Stopping the trading client...
